### Installing Libraries

In [ ]:
!pip install selenium
!pip install numpy

### Importing Libraries

In [ ]:
import time
import random
import datetime
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Setting Global Variables

In [ ]:
PATH = './bin/chromedriver.exe'
# Your username
USERNAME = ''
# Your password
PASSWORD = ''
# Course name
COURSE_NAME = ''
# Refresh rate
REFRESH_RATE = 30
# Standard deviation of the normal distribution
STD = 5

### Setting Up Session

In [ ]:
driver = webdriver.Chrome(PATH)

### Automator

###### Login

In [ ]:
driver.get('https://mis.uic.edu.hk/')
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '.logotitle'))
)
username_field = driver.find_element_by_id('uid')
username_field.send_keys(USERNAME)
password_field = driver.find_element_by_name('password')
password_field.send_keys(PASSWORD)
password_field.send_keys(Keys.RETURN)

###### Search for Course

In [ ]:
SUCCESS = False

while True:
    driver.get('https://mis.uic.edu.hk/mis/student/as/home.do')
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.tablestyle-2'))
    )
    search_field = driver.find_element_by_name('keyWord')
    search_field.send_keys(COURSE_NAME)
    search_field.send_keys(Keys.RETURN)
    table = driver.find_element_by_class_name('tablestyle-10')
    trs = table.find_elements_by_tag_name('tr')
    for idx, tr in enumerate(trs):
        if idx == 0:
            ths = tr.find_elements_by_tag_name('th')
            assert len(ths) == 8
            continue
        tds = tr.find_elements_by_tag_name('td')
        if tds[1].text == COURSE_NAME:
            btn = tds[-1].find_element_by_tag_name('input')
            course_status = btn.get_attribute('value')
            if course_status == 'Add':
            # if course_status == 'Forbid Add':
                btn.click()
                
                # Accept the alert
                driver.switch_to.alert.accept()
                
                # Check in the list of registered courses
                driver.get('https://mis.uic.edu.hk/mis/student/as/home.do')
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.tablestyle-2'))
                )
                rc_table = driver.find_element_by_class_name('tablestyle-2')
                rc_trs = rc_table.find_elements_by_tag_name('tr')
                for tc_idx, tc_tr in enumerate(rc_trs):
                    if tc_idx == 0:
                        tc_ths = tc_tr.find_elements_by_tag_name('th')
                        assert len(tc_ths) == 1
                        continue
                    
                    if tc_idx == 1:
                        tc_ths = tc_tr.find_elements_by_tag_name('th')
                        assert len(tc_ths) == 7
                        continue
                        
                    rc_tds = tc_tr.find_elements_by_tag_name('td')
                    if rc_tds[1].text == COURSE_NAME:
                        SUCCESS = True
                        course_status = 'Successfully added'
                        break
                    else:
                        course_status = 'Missed'
    
    res = '[{}][{}] {}'.format(str(datetime.datetime.now()), COURSE_NAME, course_status)
    with open('./log.txt', 'a') as log:
        log.write(res + '\n')
        
    if SUCCESS:
        print(res)
        break
        
    sleep_time = np.abs(np.random.normal(REFRESH_RATE, STD))
    print(res + '. Pausing for {}s.'.format(round(sleep_time, 2)), end = '\r')
    time.sleep(sleep_time)